In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from tqdm import tqdm
import ee
import pandas as pd
from shapely.geometry import Point,Polygon

In [4]:
try:
    ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

In [26]:
# CONSTANT
CSV_PATH    = './data/wealth_index.csv'
ALT_CSV_PATH= './data/alt_wealth_index.csv'
L8          = 'LANDSAT/LC08/C01/T1'
L7          = 'LANDSAT/LE07/C02/T1'
L5          = 'LANDSAT/LT05/C02/T1'
L8_BANDS    = ['B2','B3','B4','B5','B6','B7']
L7_BANDS    = ['B1','B2','B3','B4','B5','B7']
BAND_EQUIV  = ['BLUE','GREEN','RED','NIR','SWIR1','SWIR2']
BB_SIZE     = 0.75

In [30]:
csv = pd.read_csv(CSV_PATH)
csv.head()

,Unnamed: 0,country,year,month,day,cluster,lat,lon,households,wealthpooled,geometry,filename
0,0,angola,2011,NaN,NaN,1,-12.350257,13.534922,36,2.312757,POLYGON ((13.159921646118164 -12.7252569198608...,-12_3_13_53.tif
1,1,angola,2011,NaN,NaN,2,-12.360865,13.551494,32,2.010293,POLYGON ((13.176493644714355 -12.7358646392822...,-12_3_13_55.tif
2,2,angola,2011,NaN,NaN,3,-12.613421,13.413085,36,0.877744,POLYGON ((13.038084983825684 -12.9884214401245...,-12_6_13_41.tif
3,3,angola,2011,NaN,NaN,4,-12.581454,13.397711,35,1.066994,POLYGON ((13.022710800170898 -12.9564542770385...,-12_5_13_39.tif
4,4,angola,2011,NaN,NaN,5,-12.578135,13.418748,37,1.750153,POLYGON ((13.043747901916504 -12.9531345367431...,-12_5_13_41.tif


In [31]:
# @Yeh, 2020 
def decode_qamask(img: ee.Image) -> ee.Image:
    '''
    Args
    - img: ee.Image, Landsat 5/7/8 image containing 'pixel_qa' band
    Returns
    - masks: ee.Image, contains 5 bands of masks
    Pixel QA Bit Flags (universal across Landsat 5/7/8)
    Bit  Attribute
    0    Fill
    1    Clear
    2    Water
    3    Cloud Shadow
    4    Snow
    5    Cloud
    '''
    qa = img.select('QA_PIXEL')
    clear = qa.bitwiseAnd(2).neq(0)  # 0 = not clear, 1 = clear
    clear = clear.updateMask(clear).rename(['pxqa_clear'])

    water = qa.bitwiseAnd(4).neq(0)  # 0 = not water, 1 = water
    water = water.updateMask(water).rename(['pxqa_water'])

    cloud_shadow = qa.bitwiseAnd(8).eq(0)  # 0 = shadow, 1 = not shadow
    cloud_shadow = cloud_shadow.updateMask(cloud_shadow).rename(['pxqa_cloudshadow'])

    snow = qa.bitwiseAnd(16).eq(0)  # 0 = snow, 1 = not snow
    snow = snow.updateMask(snow).rename(['pxqa_snow'])

    cloud = qa.bitwiseAnd(32).eq(0)  # 0 = cloud, 1 = not cloud
    cloud = cloud.updateMask(cloud).rename(['pxqa_cloud'])

    masks = ee.Image.cat([clear, water, cloud_shadow, snow, cloud])
    return masks


def mask_qaclear(img: ee.Image) -> ee.Image:
    '''
    Args
    - img: ee.Image
    Returns
    - img: ee.Image, input image with cloud-shadow, snow, cloud, and unclear
        pixels masked out
    '''
    qam = decode_qamask(img)
    cloudshadow_mask = qam.select('pxqa_cloudshadow')
    snow_mask = qam.select('pxqa_snow')
    cloud_mask = qam.select('pxqa_cloud')
    return img.updateMask(cloudshadow_mask).updateMask(snow_mask).updateMask(cloud_mask)


def fill_gaps(img : ee.Image) -> ee.Image:
    return img.focal_mean(1.5,'square','pixels',2).blend(img)

In [32]:
def merge_landsat_bands(year):
    l8 = l7 = l5 = None
    start_date = str(int(year)-1)+"-01-01"
    end_date = str(int(year)+1)+"-12-31"
    if year >= 2012:
        l8 = ee.ImageCollection(L8).filterDate(start_date, end_date).filterMetadata('CLOUD_COVER', 'less_than', '10')
        l8 = l8.map(mask_qaclear).select(L8_BANDS, BAND_EQUIV)
    # if year >= 2000:
    #     l7 = ee.ImageCollection(L7).filterDate(start_date, end_date)
    #     l7 = l7.map(mask_qaclear)
    #     l7 = l7.select(L7_BANDS, BAND_EQUIV)        
    #     if l8 is not None:
    #          l8 = l8.merge(l7)
    if year <= 2011:
        l5 = ee.ImageCollection(L5).filterDate(start_date, end_date)
        l5 = l5.map(mask_qaclear).select(L7_BANDS, BAND_EQUIV)
        if l7 is not None:
            l7 = l5.merge(l7)
    if l8 is not None:
        return l8.median()
    # if l7 is not None:
    #     return l7.median()
    return L5.median()

In [33]:
def bb_from_coords(lat, lon, margin=BB_SIZE/2):
    return Polygon([
        [lon-margin, lat-margin],
        [lon+margin, lat-margin],
        [lon+margin, lat+margin],
        [lon-margin, lat+margin],
        [lon-margin, lat-margin],
    ])

def ee_bb_from_coords(lat, lon, margin=BB_SIZE/2):
    return ee.Geometry.Polygon([
        [lon-margin, lat-margin],
        [lon+margin, lat-margin],
        [lon+margin, lat+margin],
        [lon-margin, lat+margin],
        [lon-margin, lat-margin],
    ])

def add_bb_to_csv(csv, margin=BB_SIZE/2):
    csv['geometry'] = csv.apply(lambda x: bb_from_coords(x.lat, x.lon, margin), axis=1)
    return csv

In [42]:
df = pd.read_csv(CSV_PATH)
df.head()

,country,year,month,day,cluster,lat,lon,households,wealthpooled,geometry,filename
0,angola,2011,NaN,NaN,1,-12.350257,13.534922,36,2.312757,POLYGON ((13.159921646118164 -12.7252569198608...,-12_3_13_53.tif
1,angola,2011,NaN,NaN,2,-12.360865,13.551494,32,2.010293,POLYGON ((13.176493644714355 -12.7358646392822...,-12_3_13_55.tif
2,angola,2011,NaN,NaN,3,-12.613421,13.413085,36,0.877744,POLYGON ((13.038084983825684 -12.9884214401245...,-12_6_13_41.tif
3,angola,2011,NaN,NaN,4,-12.581454,13.397711,35,1.066994,POLYGON ((13.022710800170898 -12.9564542770385...,-12_5_13_39.tif
4,angola,2011,NaN,NaN,5,-12.578135,13.418748,37,1.750153,POLYGON ((13.043747901916504 -12.9531345367431...,-12_5_13_41.tif


In [36]:
csv = add_bb_to_csv(csv)
csv.head()

,country,year,month,day,cluster,lat,lon,households,wealthpooled,geometry,filename
0,angola,2011,NaN,NaN,1,-12.350257,13.534922,36,2.312757,POLYGON ((13.159921646118164 -12.7252569198608...,-12_3_13_53.tif
1,angola,2011,NaN,NaN,2,-12.360865,13.551494,32,2.010293,POLYGON ((13.176493644714355 -12.7358646392822...,-12_3_13_55.tif
2,angola,2011,NaN,NaN,3,-12.613421,13.413085,36,0.877744,POLYGON ((13.038084983825684 -12.9884214401245...,-12_6_13_41.tif
3,angola,2011,NaN,NaN,4,-12.581454,13.397711,35,1.066994,POLYGON ((13.022710800170898 -12.9564542770385...,-12_5_13_39.tif
4,angola,2011,NaN,NaN,5,-12.578135,13.418748,37,1.750153,POLYGON ((13.043747901916504 -12.9531345367431...,-12_5_13_41.tif


In [37]:
csv.to_csv(CSV_PATH, index=False)

In [25]:
for year in csv.year.unique():
    sub_csv = csv[ csv.year==year ]
    raw_image = merge_landsat_bands(int(year))
    print(year, sub_csv.country.unique())
    for country in sub_csv.country.unique():
        print(country)
        sub_csv_ = sub_csv[ sub_csv.country==country ]
        for _,row in sub_csv_.iterrows():
            region = ee_bb_from_coords(row.lat, row.lon)
            image = raw_image.clip(region)
            name = str(row.lat)[:5].replace('.','_')+"_"+str(row.lon)[:5].replace('.','_')
            task = ee.batch.Export.image.toDrive(**{
                'image': image,
                'folder': 'landsat',
                'crs' : 'EPSG:3857',
                'fileNamePrefix' : name,
                'fileFormat': 'GeoTIFF',
                'scale': 30,
                'region': region
            })
            task.start()

IndentationError: expected an indented block after 'if' statement on line 5 (3330635013.py, line 6)